In [1]:
data_root = '/home/scott/Documents/SPEC_Spider'

In [2]:
!tree /home/scott/Documents/SPEC_Spider

/home/scott/Documents/SPEC_Spider
├── cpu
│   ├── cpu2006
│   │   ├── SPECfp.csv
│   │   ├── SPECfp_rate.csv
│   │   ├── SPECint.csv
│   │   └── SPECint_rate.csv
│   └── cpu2017
│       ├── CFP2017_rate.csv
│       ├── CFP2017_speed.csv
│       ├── CINT2017_rate.csv
│       └── CINT2017_speed.csv
├── java
│   ├── jbb2015
│   │   ├── SPECjbb2015-Composite.csv
│   │   ├── SPECjbb2015-Distributed.csv
│   │   └── SPECjbb2015-MultiJVM.csv
│   └── jvm2008
│       └── jvm2008.csv
├── jbb2015.csv
└── power
    └── ssj2008.csv

7 directories, 14 files


In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
c2017_rfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_rate.csv")
c2017_sfp = pd.read_csv(f"{data_root}/cpu/cpu2017/CFP2017_speed.csv")
c2017_rint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_rate.csv")
c2017_sint = pd.read_csv(f"{data_root}/cpu/cpu2017/CINT2017_speed.csv")

c2006_rfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp_rate.csv")
c2006_sfp = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECfp.csv")
c2006_rint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint_rate.csv")
c2006_sint = pd.read_csv(f"{data_root}/cpu/cpu2006/SPECint.csv")

j2015_com = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Composite.csv")
j2015_dis = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-Distributed.csv")
j2015_mul = pd.read_csv(f"{data_root}/java/jbb2015/SPECjbb2015-MultiJVM.csv")

jvm2008 = pd.read_csv(f"{data_root}/java/jvm2008/jvm2008.csv")
ssj2008 = pd.read_csv(f"{data_root}/power/ssj2008.csv")

date: 

- cpu2017/cpu2006: `Test Date` / `HW Avail`

- jbb2015: `Test date` / `Hardware Availability`

- jvm2008: `Test date:` / `HW available`

- ssj 2008: `Test Date` / `Hardware Availability`

季度

In [114]:
month_mapper = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May':5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}
re_month_mapper = {v:k for k, v in month_mapper.items()}

re_month_mapper

{1: 'Jan',
 2: 'Feb',
 3: 'Mar',
 4: 'Apr',
 5: 'May',
 6: 'Jun',
 7: 'Jul',
 8: 'Aug',
 9: 'Sep',
 10: 'Oct',
 11: 'Nov',
 12: 'Dec'}

In [115]:
def split_date_1(item):
    """
    May-2017
    2014.3
    2010
    """
    d = 1
    if '-' in item:
        m, y = item.split('-')
    elif '.' in item:
        y, m = item.split('.')
        m = re_month_mapper[int(m)]
    elif ' ' in item:
        m, y = item.split()
    else:
        y = item
        m = 'Jan'
    m = month_mapper[m[:3]]
    y = int(y)
    return pd.Series({'day': d, 'month': m, 'year': y})

In [116]:
c2017_rfp['Test Date'].apply(lambda x: split_date_1(x))

,day,month,year
0,1,4,2022
1,1,4,2022
2,1,4,2022
3,1,4,2022
4,1,4,2022
...,...,...,...
7616,1,3,2019
7617,1,3,2019
7618,1,3,2019
7619,1,3,2019


In [121]:
def split_date_2(item):
    """
    May 1, 2018
    2009/05/19
    12.02.2009
    """
    if ',' in item:
        md, y = item.split(',')
        m, d = md.split()
    elif '/' in item:
        y, m, d = item.split('/')
        m = re_month_mapper[int(m)]
    elif '.' in item:
        d, m, y = item.split('.')
        m = re_month_mapper[int(m)]
    m = m[:3]
    if m == 'Spe':
        m = 'Sep'
    m = month_mapper[m[:3]]
    d, m, y = int(d), int(m), int(y)
    return pd.Series({'day': d, 'month': m, 'year': y})

In [122]:
j2015_com['Test date'].apply(lambda x: split_date_2(x))

,day,month,year
0,24,5,2021
1,12,10,2018
2,8,10,2018
3,8,10,2018
4,28,8,2018
...,...,...,...
170,2,7,2017
171,19,6,2017
172,26,6,2021
173,28,1,2022


In [58]:
j2015_com['Hardware Availability'].unique()

array(['May-2021', 'Nov-2018', 'Jul-2017', 'Apr-2017', 'Apr-2014',
       'Mar-2022', 'Aug-2021', 'Nov-2021', 'Apr-2021', 'May-2020',
       'Jan-2019', 'Apr-2019', 'Jul-2019', 'Dec-2018', 'Jun-2018',
       'Jan-2018', 'Nov-2017', 'Feb-2021', 'Mar-2021', 'Apr-2020',
       'December-2019', 'August-2019', 'Oct-2018', 'May-2022', 'Nov-2020',
       'Jun-2021', 'Jun-2020', 'Aug-2019', 'Sep-2017', 'Aug-2017',
       'Feb-2020', 'Jun-2019', 'Dec-2014', 'Dec-2020', 'Jan-2017',
       'May-2016', 'Mar 2016', 'Oct-2017', 'June-2021', 'April-2019',
       'August-2016', 'August-2017', 'Jul-2021'], dtype=object)

In [123]:
j2015_com['Hardware Availability'].apply(lambda x: split_date_1(x))['month'].unique()

array([ 5, 11,  7,  4,  3,  8,  1, 12,  6,  2, 10,  9])

In [124]:
def split_date_3(item):
    """ Thu Aug 20 04:16:03 GMT-05:00 2015 """
    items = item.split()
    d, m, y = items[2], items[1], items[-1]
    m = m[:3]
    m = month_mapper[m]
    d, m, y = int(d), int(m), int(y)
    return pd.Series({'day': d, 'month': m, 'year': y})

In [125]:
jvm2008['Test date:'].apply(lambda x: split_date_3(x))

,day,month,year
0,20,8,2015
1,25,12,2014
2,29,12,2011
3,8,1,2012
4,29,10,2011
5,16,9,2010
6,2,10,2009
7,5,10,2009
8,2,10,2009
9,25,3,2009


In [126]:
jvm2008['HW available'].unique()

array(['2014.3', '2014.09', '2010', 'Jan-2012', 'Oct-2011', 'Dec-2010',
       'Mar-2009', 'Apr-2009', 'Oct-2008', 'Mar-2008'], dtype=object)

In [127]:
jvm2008['HW available'].apply(lambda x: split_date_1(x))

,day,month,year
0,1,3,2014
1,1,9,2014
2,1,1,2010
3,1,1,2012
4,1,10,2011
5,1,12,2010
6,1,3,2009
7,1,3,2009
8,1,3,2009
9,1,4,2009


In [86]:
ssj2008[ssj2008['Test Date'] == 'Various']

,Suite,Test Sponsor,SPEC License #,Test Method,Tested By,Test Date,Hardware Availability,Software Availability,Publication,ssj_ops @ 100% of target load,...,JVM Instances,JVM Initial Heap (MB),JVM Maximum Heap (MB),JVM Address Bits,Boot Firmware Version,Management Firmware Version,Workload Version,Director Location,Other Software,URL Suffix
737,SPECpower_ssj2008,Acer Incorporated,97,Multi Node,Acer Incorporated,Various,Feb-2011,Dec-2009,"Apr 7, 2011","3,719,937",...,6,1950,1950,64,P08,1.20,SSJ 1.2.6,Controller,None,res2011q2/power_ssj2008-20110315-00365.html
738,SPECpower_ssj2008,Acer Incorporated,97,Multi Node,Acer Incorporated,Various,Feb-2011,Dec-2009,"Mar 23, 2011","3,732,850",...,6,1950,1950,64,P08,1.20,SSJ 1.2.6,Controller,None,res2011q1/power_ssj2008-20110308-00364.html
739,SPECpower_ssj2008,Acer Incorporated,97,Multi Node,Acer Incorporated,Various,Jan-2010,Dec-2009,"Mar 23, 2011","3,567,236",...,6,1500,1500,64,<TBD>,1.20,SSJ 1.2.6,Controller,None,res2011q1/power_ssj2008-20110304-00363.html


In [108]:
f = ssj2008['Test Date'] == 'Various'
indices = ssj2008[f].index
pub_val = ssj2008[f]['Publication']
ssj2008.loc[indices, 'Test Date'] = pub_val

In [84]:
ssj2008['Test Date'].unique()

array(['Sep 26, 2018', 'Oct 20, 2010', 'Oct 5, 2010', 'Sep 29, 2010',
       'Oct 8, 2010', 'Aug 17, 2010', 'Aug 5, 2010', 'Sep 10, 2010',
       'Sep 14, 2010', 'Aug 19, 2010', 'Aug 20, 2010', 'Jul 26, 2010',
       'May 28, 2010', 'Apr 24, 2009', 'Aug 4, 2009', 'Aug 11, 2009',
       'Sep 16, 2021', 'Sep 15, 2021', 'May 4, 2009', 'Feb 1, 2021',
       'Nov 7, 2012', 'Nov 8, 2012', 'Nov 6, 2012', 'Oct 17, 2008',
       'Oct 16, 2008', 'Dec 19, 2008', 'Aug 5, 2008', 'Aug 4, 2008',
       'Apr 22, 2021', 'May 27, 2021', 'Sep 7, 2019', 'Apr 15, 2019',
       'Dec 20, 2018', 'Jan 6, 2019', 'Aug 24, 2018', 'Jul 15, 2018',
       'Sep 25, 2014', 'Jun 28, 2012', 'Jun 19, 2012', 'Jun 12, 2012',
       'Sep 12, 2008', 'Oct 15, 2012', 'Oct 11, 2012', 'Apr 25, 2002',
       'Sep 23, 2011', 'Sep 13, 2011', 'May 26, 2012', 'May 24, 2012',
       'Nov 28, 2007', 'Jan 11, 2008', 'Nov 27, 2007', 'Jan 10, 2008',
       'Apr 20, 2009', 'Apr 17, 2009', 'Mar 10, 2009', 'Apr 23, 2018',
       'May 27, 201

In [129]:
ssj2008['Test Date'].apply(lambda x: split_date_2(x))

,day,month,year
0,26,9,2018
1,20,10,2010
2,5,10,2010
3,5,10,2010
4,29,9,2010
...,...,...,...
773,22,11,2019
774,21,11,2019
775,9,5,2019
776,11,6,2019


In [130]:
ssj2008['Hardware Availability'].unique()

array(['Sep-2018', 'Oct-2010', 'Aug-2010', 'Jun-2010', 'Jul-2010',
       'May-2009', 'Jul-2009', 'Sep-2009', 'Jul-2021', 'Jun-2009',
       'Jul-2020', 'Dec-2012', 'Dec-2008', 'Feb-2009', 'Apr-2008',
       'May-2021', 'Aug-2019', 'Apr-2019', 'Aug-2017', 'Nov-2017',
       'Jan-2015', 'Jun-2012', 'Nov-2007', 'Nov-2012', 'Nov-2011',
       'Apr-2012', 'Jul-2008', 'Feb-2018', 'Feb-2008', 'Aug-2012',
       'Feb-2011', 'Mar-2011', 'Jun-2008', 'Jul-2017', 'Jun-2017',
       'Mar-2016', 'Feb-2016', 'Sep-2014', 'Mar-2021', 'Apr-2021',
       'Sep-2008', 'Aug-2009', 'Mar-2009', 'Dec-2019', 'Aug-2020',
       'Jan-2019', 'Dec-2018', 'Oct-2017', 'Jul-2013', 'Oct-2011',
       'Jan-2011', 'May-2010', 'Sep-2010', 'Apr-2010', 'Dec-2009',
       'May-2008', 'Oct-2008', 'Jun-2021', 'Nov-2020', 'Apr-2020',
       'Jun-2020', 'Oct-2018', 'Jul-2018', 'Nov-2004', 'Mar-2008',
       'Jan-2022', 'Mar-2020', 'Jun-2019', 'Mar-2018', 'May-2018',
       'Sep-2017', 'Jun-2016', 'Jan-2008', 'Sep-2012', 'Mar-20

In [131]:
ssj2008['Hardware Availability'].apply(lambda x: split_date_1(x))

,day,month,year
0,1,9,2018
1,1,10,2010
2,1,8,2010
3,1,8,2010
4,1,10,2010
...,...,...,...
773,1,10,2019
774,1,10,2019
775,1,4,2019
776,1,4,2019


In [142]:
def get_quarter(url):
    items = url.split('/')[0]
    return pd.Series({'quarter': int(items[-1]), 'submit_year': int(items[3:7])})

In [143]:
c2017_rfp['URL Suffix'].apply(lambda x: get_quarter(x))

,quarter,submit_year
0,2,2022
1,2,2022
2,2,2022
3,2,2022
4,2,2022
...,...,...
7616,2,2019
7617,2,2019
7618,2,2019
7619,2,2019


In [140]:
c2017_rfp['URL Suffix']

0       res2022q2/cpu2017-20220411-31451.html
1       res2022q2/cpu2017-20220410-31429.html
2       res2022q2/cpu2017-20220411-31447.html
3       res2022q2/cpu2017-20220411-31449.html
4       res2022q2/cpu2017-20220410-31431.html
                        ...                  
7616    res2019q2/cpu2017-20190319-11484.html
7617    res2019q2/cpu2017-20190319-11487.html
7618    res2019q2/cpu2017-20190404-11699.html
7619    res2019q2/cpu2017-20190404-11702.html
7620    res2019q2/cpu2017-20190404-11707.html
Name: URL Suffix, Length: 7621, dtype: object